
<h3>A description of the problem to be solved</h3>

Skateboard industry averages $0.5 bln a year and sales are trending upwards (source -https://www.grandviewresearch.com/industry-analysis/skateboard-market). Skateboards alone are a popular and stable product with high demand, not even mentioning goods related to skateboarding industry (clothing, accessories etc). In my report I'm going to cover the problems that a small business owner faces when trying to open a skateboarding shop in a huge city. So the topic of my research is how to succesfully open a skateboarding store in London, with a little help from data analysis. 
In my opinion, there are 3 problems that each new skateshop owner would face:

1) **Where?** One of the toughest questions is choosing the right location. Multiple reports (https://www.researchgate.net/publication/228637298_The_Impact_of_Retail_Location_on_Retailer_Revenues_An_Empirical_Investigation) show the importance of location for success in retail industry. It's important especially in cities like London where rental and housing prices skyrocket. For skateboarding industry this question is of particular importance, as we need to take into account the most frequently visited locations of target audience (skaters)

2) **How?** It is crucial to choose what exactly differentiate us from other businesses - understanding the core value proposition for a brand. Given that there is some competition in market and open data available, it's vital to do a research and understand what people don't / do like about the existing players in a location and what we can bring to a table as a new shop

3) **What?** Chosing the right product mix. Skaters are particularly picky about brands, one of the risks of retail industry is overstocking with the brands that customers aren't willing to buy. So we need to analyze what is a customer demand for skateboarding brands in a given are and make product decision with strong data fundamentals in place

So given that the audience of this report is potential skateboard shop owners, it could be quite beneficial if we could answer those questions with data - this would allow them to avoid costly mistakes and maximize the probability of success. 

<h4>Data selection for the project.</h4>
Now that we have agreed on the problems we'll be solving, let's look at the data available. 


1. Choosing location <br>
Methodology: 
    - 1.1. One particular problem I need to solve - is locating most frequently visited locations by target audience. What are those? Skateparks of course. Once we get the skatepark data, our goal would be to find locations suitable for a new skateshop as close to these parks as possible. 
    - 1.2. To find these locations we'll be looking at existing skateshop locations in each London Borough. The goal we have here is somehow decide - does a particular borough have enough skateshops? To solve this problem we need to use of course one of the classification / regression models. We'll fit the model with the existing skateshop data and make a predictions on number of skateshops per Borough. If a predicted number is greater than current amount of skateshops in a borough - that means there is a demand in a new stores. And the opposite, if there are too many skateshops, we need to avoid those Boroughs
    - 1.3. Having Skatepark and Skateshop Model data in place we'll need to combine this data to find prospective boroughs as close to existing skateparks as possible. It might be considered as a clusterization problem, or can be solved through calculating some type of distances (like Euclidean). However, given that it's a geo data I'd like to solve it through visualization on a map. 

2. Choosing the differentiation against other skateparks.<br>
Methodology:
     - 2.1. Find some open data on customer reviews for all skateparks in london and classify them as bad / good reviews
     - 2.2. Run a simple semantic analysis (world cloud) to understand what makes a good / bad skateshop
  
3. Choosing the product mix for a skatepark.<br>
Methodology:
     - 3.1. Analyze open keyword data on most popular skateboarding brands
     - 3.2. Understand which of this brand keywords are trending by looking into user google searches 



<h2>Finding most frequently visited locations by target audience</h2>

Data description:
For a problem described in **1.1** I need to look into Foursquare skateparks data in London.
Prerequisites: to make sure that I can use them as Geo Data I need to retrieve the coordinates as well

In [22]:
# building a request url for forsquare 

RADIUS=1000
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
near='London'
category='4bf58dd8d48988d167941735'
limit=500
query='skate park'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
radius=500
LIMIT=100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&category={}&near={}&query={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    category, 
    near, 
    query,
    limit)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


In [18]:
results = requests.get(url).json()


So the data source for 1.1 looks like a data frame with skatepark name and locations

In [7]:
venues = results['response']['groups'][0]['items']  
skatepark_venues = json_normalize(venues) # flatten JSON
skatepark_venues=skatepark_venues[['venue.name','venue.location.lat','venue.location.lng']]
skatepark_venues.head()

,venue.name,venue.location.lat,venue.location.lng
0,House of Vans,51.500678,-0.113944
1,Southbank Skate Park,51.506911,-0.116636
2,Clapham Common Skatepark,51.460802,-0.143462
3,BaySixty6,51.520528,-0.204501
4,Mile End Skate Park,51.517583,-0.031460


<h2>Skateshop locations in each London Borough</h2>

part 1.2 is less trivial - as I have a goal to find 1. all the coordinates for all the boroughs 2. some data set that will supply featrues for a predictive model, something like demography data for each Borough

<h3>Table 1.2 - list of all London Boroughs with coordinates</h3>

For table 1 I'll parse London Boroughs table from wikipedia

In [25]:

# parsing wiki page into html 

fp = urllib.request.urlopen("")
mybytes = fp.read()

mystr = mybytes.decode("utf8")
fp.close()

# parsing html into BS soup object

soup = BeautifulSoup(mystr, 'html.parser')


In [26]:
# creting lists for each of the parsed table rows

table = soup.find('table', attrs={'class':'wikitable'})
table_rows = table.find_all('tr')
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.replace('\n','') for tr in td]
    l.append(row)


In [13]:
# creting header columns for each of the table headers

col_rows=table.find_all('th')

cols=[]
for th in col_rows:
    row=th.text.replace('\n','')
    cols.append(row)
cols

['Borough',
 'Inner',
 'Status',
 'Local authority',
 'Political control',
 'Headquarters',
 'Area (sq mi)',
 'Population (2013 est)[1]',
 'Co-ordinates',
 ' Nr. in map ']

In [14]:
# building a dataframe

df=pd.DataFrame(l, columns=cols)
df = df.iloc[1:]
df.head()

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2013 est)[1],Co-ordinates,Nr. in map
1,Barking and Dagenham [note 1],,,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,"194,352",51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E﻿ /...,25
2,Barnet,,,Barnet London Borough Council,Conservative,"Barnet House, 2 Bristol Avenue, Colindale",33.49,"369,088",51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W﻿ /...,31
3,Bexley,,,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,"236,687",51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E﻿ /...,23
4,Brent,,,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,"317,264",51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W﻿ /...,12
5,Bromley,,,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,"317,899",51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E﻿ /...,20


This Dataframe will be the main one to retrieve data for geo location

In [15]:
# doing some optimization of the column string content
df['Borough']=df['Borough'].str.replace(" [note]",'')
l=[]
for x in df['Borough']:
    x=x.replace(" [note]","")
    x=x.replace(" [note 1]","")
    x=x.replace(" [note 4]","")
    l.append(x)
df['Borough']=l


In [16]:
lattitude=[]
longitude=[]
for x in df['Co-ordinates']:
    z=x.index('51.') 
    z1=x.index('0.')
    if x.find('W'):
        long=0-float(x[z1:z1+6])
    else:
        long=float(x[z1:z1+6])
    lat=float(x[z:z+7])
    lattitude.append(lat)
    
    longitude.append(long)

The final dataframe with London Boroughs look like this

In [17]:
df['Longitude']=longitude
df['Lattitude']=lattitude
df.head()

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2013 est)[1],Co-ordinates,Nr. in map,Longitude,Lattitude
1,Barking and Dagenham,,,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,"194,352",51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E﻿ /...,25,-0.1557,51.5607
2,Barnet,,,Barnet London Borough Council,Conservative,"Barnet House, 2 Bristol Avenue, Colindale",33.49,"369,088",51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W﻿ /...,31,-0.1517,51.6252
3,Bexley,,,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,"236,687",51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E﻿ /...,23,-0.1505,51.4549
4,Brent,,,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,"317,264",51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W﻿ /...,12,-0.2817,51.5588
5,Bromley,,,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,"317,899",51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E﻿ /...,20,-0.0198,51.4039


<h2>Table 1.2.2. London open demography data with stats for each Borough</h2>

For a purpose of the table with features I downloaded a file from counsel website containing london census data

In [283]:
london_boroughs=pd.read_csv("/london_boroughs.csv",skiprows=0)
london_boroughs=london_boroughs[1:]
london_boroughs.head()

,Code,New code,Area name,Inner/ Outer London,GLA Population Estimate 2016,GLA Household Estimate 2016,Inland Area (Hectares),Population density (per hectare) 2016,"Average Age, 2016","Proportion of population aged 0-15, 2016","Proportion of population of working-age, 2016","Proportion of population aged 65 and over, 2016",Net internal migration (2014),Net international migration (2014),Net natural change (2014),% of resident population born abroad (2014),Largest migrant population by country of birth (2011),% of largest migrant population (2011),Second largest migrant population by country of birth (2011),% of second largest migrant population (2011),Third largest migrant population by country of birth (2011),% of third largest migrant population (2011),% of population from BAME groups (2016),% people aged 3+ whose main language is not English (2011 Census),"Overseas nationals entering the UK (NINo), (2014/15)","New migrant (NINo) rates, (2014/15)",Largest migrant population arrived during 2014/15,Second largest migrant population arrived during 2014/15,Third largest migrant population arrived during 2014/15,Employment rate (%) (2015),Male employment rate (2015),Female employment rate (2015),Unemployment rate (2015),Youth Unemployment (claimant) rate 18-24 (Dec-14),Proportion of 16-18 year olds who are NEET (%) (2014),Proportion of the working-age population who claim out-of-work benefits (%) (Aug-2015),% working-age with a disability (2015),Proportion of working age people with no qualifications (%) 2015,Proportion of working age with degree or equivalent and above (%) 2015,"Gross Annual Pay, (2015)",Gross Annual Pay - Male (2015),Gross Annual Pay - Female (2015),Modelled Household median income estimates 2012/13,% adults that volunteered in past 12 months (2010/11 to 2012/13),Number of jobs by workplace (2014),% of employment that is in public sector (2014),"Jobs Density, 2014","Number of active businesses, 2014",Two-year business survival rates (started in 2012),Crime rates per thousand population 2014/15,Fires per thousand population (2014),Ambulance incidents per hundred population (2014),"Median House Price, 2014","Average Band D Council Tax charge (£), 2015/16",New Homes (net) 2014/15 (provisional),"Homes Owned outright, (2014) %","Being bought with mortgage or loan, (2014) %","Rented from Local Authority or Housing Association, (2014) %","Rented from Private landlord, (2014) %","% of area that is Greenspace, 2005",Total carbon emissions (2013),"Household Waste Recycling Rate, 2014/15","Number of cars, (2011 Census)","Number of cars per household, (2011 Census)","% of adults who cycle at least once per month, 2013/14","Average Public Transport Accessibility score, 2014","Achievement of 5 or more A*- C grades at GCSE or equivalent including English and Maths, 2013/14",Rates of Children Looked After (2015),% of pupils whose first language is not English (2015),% children living in out-of-work households (2014),"Male life expectancy, (2012-14)","Female life expectancy, (2012-14)",Teenage conception rate (2014),Life satisfaction score 2011-14 (out of 10),Worthwhileness score 2011-14 (out of 10),Happiness score 2011-14 (out of 10),Anxiety score 2011-14 (out of 10),Childhood Obesity Prevalance (%) 2014/15,People aged 17+ with diabetes (%),Mortality rate from causes considered preventable 2012/14,Political control in council,Proportion of seats won by Conservatives in 2014 election,Proportion of seats won by Labour in 2014 election,Proportion of seats won by Lib Dems in 2014 election,Turnout at 2014 local elections
1,E09000001,E09000001,City of London,Inner London,"8,548","5,179",290.4,28.9,42.9,27.2,90.6,9.4,138,252,35,.,United States,2.8,France,2.0,Australia,1.9,27.5,17.1,892,151.0,France,United States,India,64.6,.,.,.,1.2,.,3.8,.,.,.,.,.,.,"£99,390",.,"500,400",3.4,84.60,"19,250",63.0,.,12.3,.,"765,000",943,230,.,.,.,.,4.8,"1,417.5",34.4,"1,692",0.4,.,7.9,78.6,84,.,9.1,.,.,.,6.59,7.08,5.99,5.57,NaN,2.6,128.8,.,.,.,.,.
2,E09

<h2>Table 2.1.Open data on customer reviews for all skateparks in London</h2>
<p>For a purpose of this task I just downloaded the data of skateboarding shop reivews from google maps and stored it in csv</p>

In [19]:
df=pd.read_csv('/Skateshop.csv')
df.head()

,Mark,Review
0,4,"The staff were friendly and very helpful, and ..."
1,4,My only advice is to check the website before ...
2,3,"The guy who assisted me was very nice, patient..."
3,5,Super helpful and super nice guys. Got a custo...
4,2,Went into the store yesterday. Arrogant and un...


<h2>Table 3. Keyword data on most popular skateboarding brands</h2>
<p>For a purpose of this task parsed the list of popular skate brands</p>

In [ ]:
fp = urllib.request.urlopen()
mybytes = fp.read()

mystr = mybytes.decode("utf8")
fp.close()

In [20]:
# some of the results I got
selector = ['adidas',
 'Anti Hero',
 'Antix',
 'Anuell',
 'Baker',
 'Carhartt WIP',
 'Carpet Company',
 'Chocolate',
 'Converse',
 'DC',
 'Dickies',
 'Element',
 'Emerica',
 'Enjoi',
 'Etnies',
 'Flip',
 'Girl',
 'GX1000']

['adidas',
 'Anti Hero',
 'Antix',
 'Anuell',
 'Baker',
 'Carhartt WIP',
 'Carpet Company',
 'Chocolate',
 'Converse',
 'DC',
 'Dickies',
 'Element',
 'Emerica',
 'Enjoi',
 'Etnies',
 'Flip',
 'Girl',
 'GX1000']

Then we can user Google ads keyword planner API to get results on which of them have most of traffic 

In [ ]:
# best keyword estimate



offset = 0
selector['paging'] = {
    'startIndex': str(offset),
    'numberResults': str(PAGE_SIZE)
}


page = targeting_idea_service.get(selector)

for result in page['entries']:
  attributes = {}
  for attribute in result['data']:
    attributes[attribute['key']] = getattr(
        attribute['value'], 'value', '0')
  print('Keyword with "%s" text and average monthly search volume '
        '"%s" was found with Products and Services categories: %s.'
        % (attributes['KEYWORD_TEXT'],
            attributes['SEARCH_VOLUME'],
            attributes['CATEGORY_PRODUCTS_AND_SERVICES']))    
